In [1]:
import os
import re
import pandas as pd
from dbfread import DBF
import tqdm
inicio=os.getcwd()

El primer problema que encontramos trabajando con la base de datos de SUNAT, es que son varios archivos en formato dbf. A continuacion, esta el procedimiento que seguimos para leer estos archivos con pandas, y luego transformarlos a csv (bajo un archivo único).

Las tablas que se crearan de estas bases (las más grandes), son:
- Importación (tipo A)
- Importación (tipo B)
- Exportación

### Transformacion: dfs -> csv

aca estoy descomprimiendo los archivos

In [ ]:
from tqdm import tqdm
# Este codigo ya no funciona, porque recibía una relación de todos los archivos con la base de SUNAT

e='mb'
df_mam=pd.DataFrame()
arc_dcd=[os.path.join(inicio,f'{e}\\{i}') for i in os.listdir(os.path.join(inicio,e))]
for a in tqdm(arc_dcd):
    table= DBF(a, load=True, encoding='latin1')
    df = pd.DataFrame(iter(table))
    df_mam=pd.concat([df_mam, df],axis=0,ignore_index=True)
df_mam.shape


In [12]:
# Esta linea era para guardar los nuevos archivos
df_mam.to_csv('ma.csv',sep=';',index=False)

El resultado de esta parte son los archivos que comprenden el grueso de nuestra [base](https://drive.google.com/drive/folders/1285u4JRQz8C6PBzDluSs2QSMNhT8oiHz?usp=sharing)

# SQL

In [13]:
import os
import re
import pandas as pd
import pyodbc
from tqdm import tqdm

In [2]:
conn = pyodbc.connect(
    "Driver={SQL Server};"
    "Server=QMIN;"
    "database=borradorBI;"
    "Trusted_Connection = yes;"
)
cursor = conn.cursor()

## Creacion de tablas

In [3]:
# Tabla de ImportacionA (ImportacionesA)

cursor.execute("""
IF OBJECT_ID('ImportacionesA', 'U') IS NULL
BEGIN
    CREATE TABLE ImportacionesA (
        CODI_ADUAN VARCHAR(3) NOT NULL,
        ANO_PRESE VARCHAR(2) NOT NULL,
        NUME_CORRE VARCHAR(6) NOT NULL,
        FECH_INGSI NUMERIC(8),
        TIPO_DOCUM VARCHAR(1),
        LIBR_TRIBU VARCHAR(11),
        DNOMBRE VARCHAR(40),
        CODI_AGENT VARCHAR(4),
        FECH_LLEGA NUMERIC(8),
        VIA_TRANSP NUMERIC(1),
        EMPR_TRANS VARCHAR(4),
        CODI_ALMA VARCHAR(4),
        CADU_MANIF VARCHAR(3),
        FECH_MANIF VARCHAR(4),
        NUME_MANIF VARCHAR(6),
        FECH_RECEP NUMERIC(8),
        FECH_CANCE NUMERIC(8),
        TIPO_CANCE NUMERIC(2),
        BANC_CANCE VARCHAR(3),
        CODI_ENFIN VARCHAR(3),
        DK VARCHAR(1),
        PAIS_ORIGE VARCHAR(3),
        PAIS_ADQUI VARCHAR(3),
        PUER_EMBAR VARCHAR(6),
        NUME_SERIE VARCHAR(4) NOT NULL,
        PART_NANDI NUMERIC(10),
        DESC_COMER VARCHAR(90),
        DESC_MATCO VARCHAR(90),
        DESC_USOAP VARCHAR(90),
        DESC_FOPRE VARCHAR(90),
        DESC_OTROS VARCHAR(90),
        FOB_DOLPOL NUMERIC(14,3),
        FLE_DOLAR NUMERIC(11,3),
        SEG_DOLAR NUMERIC(11,3),
        PESO_NETO NUMERIC(14,3),
        PESO_BRUTO NUMERIC(14,3),
        UNID_FIQTY NUMERIC(12,3),
        UNID_FIDES VARCHAR(3),
        QUNICOM NUMERIC(14,3),
        TUNICOM VARCHAR(3),
        SEST_MERCA VARCHAR(5),
        ADV_DOLAR NUMERIC(11,2),
        IGV_DOLAR NUMERIC(11,2),
        ISC_DOLAR NUMERIC(11,2),
        IPM_DOLAR NUMERIC(11,2),
        DES_DOLAR NUMERIC(11,2),
        IPA_DOLAR NUMERIC(11,2),
        SAD_DOLAR NUMERIC(11,2),
        DER_ADUM NUMERIC(11,2),
        COMM NUMERIC(11,2),
        FMOD NUMERIC(8),
        CANT_BULTO NUMERIC(14,3),
        CLASE VARCHAR(3),
        TRAT_PREFE NUMERIC(3),
        TIPO_TRAT NUMERIC(1),
        CODI_LIBER NUMERIC(4),
        IMPR_RELIQ NUMERIC(4)
    );
END
""")

conn.commit()


In [4]:
# Tabla de ImportacionB (ImportacionB)

cursor.execute("""
IF OBJECT_ID('ImportacionB', 'U') IS NULL
BEGIN
    CREATE TABLE ImportacionB (
        CODI_ADUAN VARCHAR(3) NOT NULL, -- Código de aduana
        ANO_PRESE VARCHAR(2) NOT NULL, -- Año de numeración de la DUA
        NUME_CORRE VARCHAR(6) NOT NULL, -- Número de DUA
        NUME_SECUP NUMERIC(4), -- Número de secuencia del proveedor
        FECH_INGSI NUMERIC(8), -- Fecha de ingreso de la póliza
        CODI_PROVE VARCHAR(10), -- Código del proveedor
        NOMB_PROVE VARCHAR(60), -- Nombre del proveedor
        PART_NANDI NUMERIC(10), -- Partida arancelaria
        NUME_ITEM VARCHAR(4), -- Número de serie
        VFOB_ITEM NUMERIC(17,6), -- FOB en dólares (FOB unitario x cantidad)
        NOMB_COMER VARCHAR(30), -- Nombre comercial
        MARC_COMER VARCHAR(25), -- Marca comercial
        MODE_MERCD VARCHAR(30), -- Modelo
        CANT_MERCD NUMERIC(17,6), -- Cantidad
        UNID_MERCD VARCHAR(3), -- Unidad de medida
        CARA_TIPO VARCHAR(90), -- Característica del producto
        PAIS_ORIGE VARCHAR(2), -- País de origen
        ARO_ANO VARCHAR(4), -- Año de producción
        ESTA_MERCD VARCHAR(2), -- Estado del producto
        VFOB_UNITA NUMERIC(17,6) -- FOB unitario
    );
END
""")

conn.commit()


In [18]:
# Tabla de Exportacion (Exportaciones)

cursor.execute("""
IF OBJECT_ID('Exportaciones', 'U') IS NULL
BEGIN
    CREATE TABLE Exportaciones (
        CADU VARCHAR(3) NOT NULL, -- Código de aduana
        FANO VARCHAR(4) NOT NULL, -- Fecha de numeración de la orden de embarque
        NDCL VARCHAR(6) NOT NULL, -- Número de orden de embarque
        FNUM NUMERIC(8), -- Fecha de numeración de la orden de embarque
        FEMB NUMERIC(8), -- Fecha de embarque
        FECH_RECEP NUMERIC(8), -- Fecha de recepción de la declaración
        NDCLREG VARCHAR(6) NOT NULL, -- Número de la declaración de exportación
        FREG NUMERIC, -- Fecha de regularización
        FANOREG VARCHAR(4), -- Año de regularización de exportación
        CAGE VARCHAR(4), -- Código de agente de aduana
        TDOC VARCHAR(1), -- Tipo de documento del exportador
        NDOC VARCHAR(11), -- Número de documento del exportador
        DNOMBRE VARCHAR(40), -- Razón social del exportador
        CPAIDES VARCHAR(3), -- Código de país de destino
        CPUEDES VARCHAR(6), -- Código del puerto de destino
        CVIATRA VARCHAR(1), -- Código de la vía de transporte
        CUNITRA VARCHAR(1), -- Código de unidad de transporte
        CEMPTRA VARCHAR(4), -- Código de empresa de transporte
        DMAT VARCHAR(30), -- Matrícula de la nave
        NCON VARCHAR(20), -- Número de conocimiento
        CENTFIN VARCHAR(3), -- Código de entidad financiera
        CALM VARCHAR(4), -- Código de almacén
        DNOMPRO VARCHAR(60), -- Nombre del proveedor
        DDIRPRO VARCHAR(80), -- Dirección del proveedor
        DK VARCHAR(1), -- Indicador de teledespacho para O.E.
        DK2 VARCHAR(1), -- Indicador de teledespacho para DUE
        NSER VARCHAR(4) NOT NULL, -- Número de serie
        PART_NANDI NUMERIC(10), -- Código de la partida nandina
        DCOM VARCHAR(100), -- Descripción comercial
        DMER2 VARCHAR(100), -- Descripción comercial
        DMER3 VARCHAR(100), -- Descripción comercial
        DMER4 VARCHAR(100), -- Descripción comercial
        DMER5 VARCHAR(100), -- Descripción comercial
        VFOBSERDOL NUMERIC(11,3), -- Valor FOB de la serie
        VPESNET NUMERIC(14,3), -- Peso neto de la serie
        VPESBRU NUMERIC(14,3), -- Peso bruto de la serie
        QUNIFIS NUMERIC(14,3), -- Cantidad exportada
        TUNIFIS VARCHAR(10), -- Unidad de medida
        QUNICOM NUMERIC(14,3), -- Cantidad de unidad comercial
        TUNICOM VARCHAR(3), -- Tipo de unidad comercial
        UBIGEO VARCHAR(6) -- Código de ubicación geográfica
    );
END
""")

conn.commit()


Las siguientes son tablas que contienen los recursos para algunas abreviaciones (paises, puertos, etc.)

In [ ]:
# Tablas adicionales 

cursor.execute("""
IF OBJECT_ID('exp', 'U') IS NULL
BEGIN
    CREATE TABLE exp (
        CODI_ADUAN VARCHAR(3) NOT NULL,
        ANO_PRESE VARCHAR(2) NOT NULL,
        NUME_CORRE VARCHAR(6) NOT NULL,
        FECH_INGSI NUMERIC(8),
        TIPO_DOCUM VARCHAR(1),
        LIBR_TRIBU VARCHAR(11),
    );
END
""")

conn.commit()


## Inserción de información

### ImportacionesA

Primero realizaremos un pre procesamiento de la información breve, Luego el llenado de las tablas

In [15]:
# Cargamos los datos de los archivos ma e ImportacionA (mismo contenido)
df_ma = pd.read_csv('ImportacionA.csv',encoding='utf-8', sep=';')
df_mam = pd.read_csv('mam.csv',encoding='utf-8', sep=';')

C:\Users\gonza\AppData\Local\Temp\ipykernel_20860\597720137.py:3: DtypeWarning: Columns (4,5,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mam = pd.read_csv('mam.csv',encoding='utf-8', sep=';')
C:\Users\gonza\AppData\Local\Temp\ipykernel_20860\597720137.py:6: DtypeWarning: Columns (4,5,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mam2 = pd.read_csv('mam2.csv',encoding='utf-8', sep=';')


In [16]:
# Unimos ambas tablas en una unica de importaciones 
df_importaciones=pd.concat([df_ma,df_mam],axis=0)
df_importaciones.shape

(7941289, 58)

In [17]:
print("Campos con nan")
for c in df_importaciones.columns:
    x=sum(df_importaciones[c].isna())
    if x!=0:
        print(c, x)

Campos con nan
DNOMBRE 6
EMPR_TRANS 672283
BANC_CANCE 3032787
CODI_ENFIN 5876107
PAIS_ORIGE 3
PUER_EMBAR 675
DESC_MATCO 437412
DESC_USOAP 760168
DESC_FOPRE 46294
DESC_OTROS 1778427
IMPR_RELIQ 6790055


In [18]:
# Eliminamos duplicados y reemplazamos nan (ocasionan problemas al momento de subir la base)
df_importaciones=df_importaciones.drop_duplicates()
df_importaciones=df_importaciones.fillna(0)

In [19]:
df_importaciones.head()

,CODI_ADUAN,ANO_PRESE,NUME_CORRE,FECH_INGSI,TIPO_DOCUM,LIBR_TRIBU,DNOMBRE,CODI_AGENT,FECH_LLEGA,VIA_TRANSP,...,SAD_DOLAR,DER_ADUM,COMM,FMOD,CANT_BULTO,CLASE,TRAT_PREFE,TIPO_TRAT,CODI_LIBER,IMPR_RELIQ
0,172,24,14617,20240905,4,20602049451,"""IMPORTACIONES MUSSA EMPRESA INDIVIDUAL",7521,20240906,7,...,0.0,0.0,0.0,20240906,343.0,CAJ,0,1,0,0.0
1,172,24,14617,20240905,4,20602049451,"""IMPORTACIONES MUSSA EMPRESA INDIVIDUAL",7521,20240906,7,...,0.0,0.0,0.0,20240906,49.0,CAJ,0,1,0,0.0
2,172,24,14617,20240905,4,20602049451,"""IMPORTACIONES MUSSA EMPRESA INDIVIDUAL",7521,20240906,7,...,0.0,0.0,0.0,20240906,98.0,CAJ,0,1,0,0.0
3,172,24,14617,20240905,4,20602049451,"""IMPORTACIONES MUSSA EMPRESA INDIVIDUAL",7521,20240906,7,...,0.0,0.0,0.0,20240906,98.0,CAJ,0,1,0,0.0
4,172,24,14617,20240905,4,20602049451,"""IMPORTACIONES MUSSA EMPRESA INDIVIDUAL",7521,20240906,7,...,0.0,0.0,0.0,20240906,640.0,CAJ,0,1,0,0.0


In [20]:
try:
    for index, row in tqdm(df_importaciones.iterrows(),total=len(df_importaciones), desc="Procesando registros"):
        # Revisa la longitud de las columnas y valores para identificar el problema
        columnas = [
            'CODI_ADUAN', 'ANO_PRESE', 'NUME_CORRE', 'FECH_INGSI', 'TIPO_DOCUM', 'LIBR_TRIBU',
            'DNOMBRE', 'CODI_AGENT', 'FECH_LLEGA', 'VIA_TRANSP', 'EMPR_TRANS', 'CODI_ALMA',
            'CADU_MANIF', 'FECH_MANIF', 'NUME_MANIF', 'FECH_RECEP', 'FECH_CANCE', 'TIPO_CANCE',
            'BANC_CANCE', 'CODI_ENFIN', 'DK', 'PAIS_ORIGE', 'PAIS_ADQUI', 'PUER_EMBAR',
            'NUME_SERIE', 'PART_NANDI', 'DESC_COMER', 'DESC_MATCO', 'DESC_USOAP', 'DESC_FOPRE',
            'DESC_OTROS', 'FOB_DOLPOL', 'FLE_DOLAR', 'SEG_DOLAR', 'PESO_NETO', 'PESO_BRUTO',
            'UNID_FIQTY', 'UNID_FIDES', 'QUNICOM', 'TUNICOM', 'SEST_MERCA', 'ADV_DOLAR',
            'IGV_DOLAR', 'ISC_DOLAR', 'IPM_DOLAR', 'DES_DOLAR', 'IPA_DOLAR', 'SAD_DOLAR',
            'DER_ADUM', 'COMM', 'FMOD', 'CANT_BULTO', 'CLASE', 'TRAT_PREFE', 'TIPO_TRAT',
            'CODI_LIBER', 'IMPR_RELIQ'
        ]
        
        valores = [
            row['CODI_ADUAN'], row['ANO_PRESE'], row['NUME_CORRE'], row['FECH_INGSI'],
            row['TIPO_DOCUM'], row['LIBR_TRIBU'], row['DNOMBRE'], row['CODI_AGENT'],
            row['FECH_LLEGA'], row['VIA_TRANSP'], row['EMPR_TRANS'], row['CODI_ALMA'],
            row['CADU_MANIF'], row['FECH_MANIF'], row['NUME_MANIF'], row['FECH_RECEP'],
            row['FECH_CANCE'], row['TIPO_CANCE'], row['BANC_CANCE'], row['CODI_ENFIN'],
            row['DK'], row['PAIS_ORIGE'], row['PAIS_ADQUI'], row['PUER_EMBAR'],
            row['NUME_SERIE'], row['PART_NANDI'], row['DESC_COMER'], row['DESC_MATCO'],
            row['DESC_USOAP'], row['DESC_FOPRE'], row['DESC_OTROS'], row['FOB_DOLPOL'],
            row['FLE_DOLAR'], row['SEG_DOLAR'], row['PESO_NETO'], row['PESO_BRUTO'],
            row['UNID_FIQTY'], row['UNID_FIDES'], row['QUNICOM'], row['TUNICOM'],
            row['SEST_MERCA'], row['ADV_DOLAR'], row['IGV_DOLAR'], row['ISC_DOLAR'],
            row['IPM_DOLAR'], row['DES_DOLAR'], row['IPA_DOLAR'], row['SAD_DOLAR'],
            row['DER_ADUM'], row['COMM'], row['FMOD'], row['CANT_BULTO'], row['CLASE'],
            row['TRAT_PREFE'], row['TIPO_TRAT'], row['CODI_LIBER'], row['IMPR_RELIQ']
        ]
        
        if len(columnas) != len(valores):
            print(f"Discrepancia: {len(columnas)} columnas, {len(valores)} valores")
            break  

        try:
            cursor.execute(f"""
                INSERT INTO ImportacionesA (
                    {", ".join(columnas)}
                ) VALUES ({", ".join(["?" for _ in columnas])})
            """, valores)
        except Exception as e:
            print(f"Error al insertar el registro en el índice {index}: {e}")
            for col, val in zip(columnas, valores):
                try:
                    cursor.execute(f"SELECT CAST(? AS VARCHAR(255))", (val,))
                except Exception as col_error:
                    print(f"Problema en la columna '{col}' con valor '{val}': {col_error}")
                    break  

    conn.commit()
except ValueError as e:
    print("Error general al procesar los registros:", e)


Procesando registros: 100%|██████████| 7141397/7141397 [2:39:01<00:00, 748.43it/s]   


### Importaciones B

In [5]:
df_mb = pd.read_csv('ImportacionB.csv',encoding='utf-8', sep=';')

In [6]:
df_mb.columns

Index(['CODI_ADUAN', 'ANO_PRESE', 'NUME_CORRE', 'NUME_SECUP', 'NUME_SECUF',
       'FECH_INGSI', 'CODI_PROVE', 'NOMB_PROVE', 'PART_NANDI', 'NUME_ITEM',
       'VFOB_ITEM', 'NOMB_COMER', 'MARC_COMER', 'MODE_MERCD', 'CANT_MERCD',
       'UNID_MERCD', 'CARA_TIPO', 'PAIS_ORIGE', 'ARO_ANO', 'ESTA_MERCD',
       'VFOB_UNITA', 'TERM_TRANS', 'LUGA_TRANS'],
      dtype='object')

In [7]:
print("Campos con nan")
for c in df_mb.columns:
    x=sum(df_mb[c].isna())
    if x!=0:
        print(c, x)

Campos con nan
NOMB_COMER 27498
MARC_COMER 69
MODE_MERCD 1621
CARA_TIPO 8625
PAIS_ORIGE 2
ARO_ANO 1893


In [8]:
# Eliminamos duplicados y reemplazamos nan (ocasionan problemas al momento de subir la base)
df_mb=df_mb.drop_duplicates()
df_mb=df_mb.fillna(0)

In [9]:
df_mb.head()

,CODI_ADUAN,ANO_PRESE,NUME_CORRE,NUME_SECUP,NUME_SECUF,FECH_INGSI,CODI_PROVE,NOMB_PROVE,PART_NANDI,NUME_ITEM,...,MODE_MERCD,CANT_MERCD,UNID_MERCD,CARA_TIPO,PAIS_ORIGE,ARO_ANO,ESTA_MERCD,VFOB_UNITA,TERM_TRANS,LUGA_TRANS
0,19,24,18,1,1,20240904,No Disponi,No Disponible,8430490000,1,...,S/M,1.0,U,"DE PERFORACION PETROLERA ,USADO,DESARMADO,REFE...",IT,0.0,22,6856211.860,FCA,HUAQUILLAS
1,19,24,18,1,2,20240904,No Disponi,No Disponible,8429510000,2,...,980 G,1.0,U,MONTACARGA (CARGADOR FRONTAL) *9CM01332*,US,0.0,20,106425.000,FCA,HUAQUILLAS
2,19,24,18,1,2,20240904,No Disponi,No Disponible,8431490000,3,...,S/M,2.0,U,INCLUYE ACCESORIOS Y PARTES PARA SU NORMAL FUN...,US,0.0,20,4025.135,FCA,HUAQUILLAS
3,19,24,18,1,3,20240904,No Disponi,No Disponible,8430490000,4,...,S/M,1.0,U,ONE NEW POWER CONTROL ROOM-SCR FOR HH220,NL,0.0,20,835058.180,FCA,HUAQUILLAS
4,19,24,18,1,4,20240904,No Disponi,No Disponible,8430490000,5,...,9T1000,1.0,U,MUD PUMPS 9T1000DC-9T1000-155B,IT,0.0,20,451023.260,FCA,HUAQUILLAS


In [14]:
try:
    for index, row in tqdm(df_mb.iterrows(), total=len(df_mb), desc= "Procesando registros"):
        # Revisa la longitud de las columnas y valores para identificar el problema
        columnas = [
           'CODI_ADUAN', 'ANO_PRESE', 'NUME_CORRE', 'NUME_SECUP',
           'FECH_INGSI', 'CODI_PROVE', 'NOMB_PROVE', 'PART_NANDI', 'NUME_ITEM',
           'VFOB_ITEM', 'NOMB_COMER', 'MARC_COMER', 'MODE_MERCD', 'CANT_MERCD',
           'UNID_MERCD', 'CARA_TIPO', 'PAIS_ORIGE', 'ARO_ANO', 'ESTA_MERCD',
           'VFOB_UNITA'
        ]
        
        valores = [
            row['CODI_ADUAN'], row['ANO_PRESE'], row['NUME_CORRE'],
            row['NUME_SECUF'], row['FECH_INGSI'], row['CODI_PROVE'], row['NOMB_PROVE'],
            row['PART_NANDI'], row['NUME_ITEM'], row['VFOB_ITEM'], row['NOMB_COMER'],
            row['MARC_COMER'], row['MODE_MERCD'], row['CANT_MERCD'], row['UNID_MERCD'],
            row['CARA_TIPO'], row['PAIS_ORIGE'], row['ARO_ANO'], row['ESTA_MERCD'],
            row['VFOB_UNITA']
]

        
        # Asegúrate de que las longitudes coincidan
        if len(columnas) != len(valores):
            print(f"Discrepancia: {len(columnas)} columnas, {len(valores)} valores")
            break  

        try:
            # Inserta el registro en la base de datos
            cursor.execute(f"""
                INSERT INTO ImportacionB (
                    {", ".join(columnas)}
                ) VALUES ({", ".join(["?" for _ in columnas])})
            """, valores)
        except Exception as e:
            # Imprime el índice del registro, la columna y el valor problemático
            print(f"Error al insertar el registro en el índice {index}: {e}")
            for col, val in zip(columnas, valores):
                try:
                    # Prueba insertar cada campo individualmente para aislar el valor problemático
                    cursor.execute(f"SELECT CAST(? AS VARCHAR(255))", (val,))
                except Exception as col_error:
                    print(f"Problema en la columna '{col}' con valor '{val}': {col_error}")
                    break  # Sal del loop una vez encontrado el valor problemático

    conn.commit()
except ValueError as e:
    print("Error general al procesar los registros:", e)



Procesando registros:   0%|          | 0/2543947 [00:00<?, ?it/s]

Procesando registros: 100%|██████████| 2543947/2543947 [11:36<00:00, 3650.38it/s] 


### Exportaciones

In [46]:
df_exp = pd.read_csv('Exportacion.csv',encoding='utf-8', sep=';')

C:\Users\gonza\AppData\Local\Temp\ipykernel_16820\3645491519.py:1: DtypeWarning: Columns (18,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_exp = pd.read_csv('Exportacion.csv',encoding='utf-8', sep=';')


In [47]:
df_exp.columns

Index(['CADU', 'FANO', 'NDCL', 'FNUM', 'FEMB', 'FECH_RECEP', 'NDCLREG', 'FREG',
       'FANOREG', 'CAGE', 'TDOC', 'NDOC', 'DNOMBRE', 'CPAIDES', 'CPUEDES',
       'CVIATRA', 'CUNITRA', 'CEMPTRA', 'DMAT', 'NCON', 'CENTFIN', 'CALM',
       'DNOMPRO', 'DDIRPRO', 'DK', 'DK2', 'NSER', 'PART_NANDI', 'DCOM',
       'DMER2', 'DMER3', 'DMER4', 'DMER5', 'CEST', 'VFOBSERDOL', 'VPESNET',
       'VPESBRU', 'QUNIFIS', 'TUNIFIS', 'QUNICOM', 'TUNICOM', 'UBIGEO',
       'DNOMCON', 'DDIRCON'],
      dtype='object')

In [48]:
print("Campos con nan")
for c in df_exp.columns:
    x=sum(df_exp[c].isna())
    if x!=0:
        print(c, x)

Campos con nan
NDCLREG 4562
FANOREG 4562
CAGE 614
DNOMBRE 17724
CPAIDES 17
CPUEDES 36167
CUNITRA 334692
CEMPTRA 58262
DMAT 334692
NCON 47471
CENTFIN 317239
CALM 49948
DNOMPRO 5
DDIRPRO 5
DK 334692
DK2 334692
DMER2 2
DMER3 7
DMER4 19
DMER5 41
UBIGEO 16
DNOMCON 334769
DDIRCON 334769


Observamos nulos en el numero de declaracion de la exportación. **Esto no deberia ser posible**. Aclaramos errores con la base original, y ajustamos el df para cumplir con las reglas de la base

In [51]:
# Eliminamos duplicados y reemplazamos nan (ocasionan problemas al momento de subir la base)
df_exp.dropna(subset=['NDCLREG'], inplace=True)
df_exp.drop_duplicates(inplace=True)
df_exp.fillna(0, inplace=True)

In [52]:
df_exp.head()

,CADU,FANO,NDCL,FNUM,FEMB,FECH_RECEP,NDCLREG,FREG,FANOREG,CAGE,...,VFOBSERDOL,VPESNET,VPESBRU,QUNIFIS,TUNIFIS,QUNICOM,TUNICOM,UBIGEO,DNOMCON,DDIRCON
0,118,2024,91488,20240829,20240902,20240829,91488.0,20240906,2024.0,6437.0,...,3240.0,18000.0,18000.0,18000.0,KG,18.0,TM,150118.0,0.0,0.0
1,118,2024,91488,20240829,20240902,20240829,91488.0,20240906,2024.0,6437.0,...,1709.4,8400.0,8690.0,8400.0,KG,462.0,U,150118.0,0.0,0.0
2,118,2024,91488,20240829,20240902,20240829,91488.0,20240906,2024.0,6437.0,...,4752.0,26400.0,26400.0,26400.0,KG,26.4,TM,150118.0,0.0,0.0
3,118,2024,91488,20240829,20240902,20240829,91488.0,20240906,2024.0,6437.0,...,4752.0,26400.0,26400.0,26400.0,KG,26.4,TM,150118.0,0.0,0.0
4,118,2024,87967,20240822,20240826,20240822,87967.0,20240902,2024.0,6437.0,...,3456.0,19200.0,19200.0,19200.0,KG,19.2,TM,150118.0,0.0,0.0


In [53]:
try:
    for index, row in tqdm(df_exp.iterrows()):
        # Revisa la longitud de las columnas y valores para identificar el problema
        columnas = [
           'CADU', 'FANO', 'NDCL', 'FNUM', 'FEMB', 'FECH_RECEP', 'NDCLREG', 'FREG',
           'FANOREG', 'CAGE', 'TDOC', 'NDOC', 'DNOMBRE', 'CPAIDES', 'CPUEDES',
           'CVIATRA', 'CUNITRA', 'CEMPTRA', 'DMAT', 'NCON', 'CENTFIN', 'CALM',
           'DNOMPRO', 'DDIRPRO', 'DK', 'DK2', 'NSER', 'PART_NANDI', 'DCOM',
           'DMER2', 'DMER3', 'DMER4', 'DMER5', 'VFOBSERDOL', 'VPESNET',
           'VPESBRU', 'QUNIFIS', 'TUNIFIS', 'QUNICOM', 'TUNICOM', 'UBIGEO'
        ]
        
        valores = [
            row['CADU'], row['FANO'], row['NDCL'], row['FNUM'],
            row['FEMB'], row['FECH_RECEP'], row['NDCLREG'], row['FREG'],
            row['FANOREG'], row['CAGE'], row['TDOC'], row['NDOC'],
            row['DNOMBRE'], row['CPAIDES'], row['CPUEDES'], row['CVIATRA'],
            row['CUNITRA'], row['CEMPTRA'], row['DMAT'], row['NCON'],   
            row['CENTFIN'], row['CALM'], row['DNOMPRO'], row['DDIRPRO'],
            row['DK'], row['DK2'], row['NSER'], row['PART_NANDI'],
            row['DCOM'], row['DMER2'], row['DMER3'], row['DMER4'],
            row['DMER5'], row['VFOBSERDOL'], row['VPESNET'], row['VPESBRU'],
            row['QUNIFIS'], row['TUNIFIS'], row['QUNICOM'], row['TUNICOM'],
            row['UBIGEO']
]

        if len(columnas) != len(valores):
            print(f"Discrepancia: {len(columnas)} columnas, {len(valores)} valores")
            break  

        try:
            cursor.execute(f"""
                INSERT INTO Exportaciones (
                    {", ".join(columnas)}
                ) VALUES ({", ".join(["?" for _ in columnas])})
            """, valores)
        except Exception as e:
            print(f"Error al insertar el registro en el índice {index}: {e}")
            for col, val in zip(columnas, valores):
                try:
                    cursor.execute(f"SELECT CAST(? AS VARCHAR(255))", (val,))
                except Exception as col_error:
                    print(f"Problema en la columna '{col}' con valor '{val}': {col_error}")
                    break  

    conn.commit()
except ValueError as e:
    print("Error general al procesar los registros:", e)

